Protocollo Besmart UDP
Dettaglio specifiche







Librerie Typescript : 

https://github.com/keichi/binary-parser

https://github.com/anfema/bin-grammar

https://www.npmjs.com/package/@astronautlabs/bitstream?activeTab=readme --> Piu promettente // non gestisce little-endian!



MSG_ID (https://github.com/dianlight/BeSIM-MQTT/blob/e1f0dadc6a86ebdf44f353c8271ec51c6ab5897c/udpserver.py)
    0x02    SET_MODE
    0x0A    PROGRAM ( 1/day )
    0x0B    SET_T3
    0x0C    SET_T2
    0x0D    SET_T1
    0x11    - Unknown
    0x12    ADVANCE 
    0x15    SW_VERSION
    0x16    SET_CURVE ( OpenTherm - °C * 10)
    0x17    SET_MIN_HEAT_SETP  ( Ophenterm )
    0x18    SET_MAX_HEAT_SETP  ( Ophenterm )
    0x19    SET_UNITS (0=°C 1=°F)
    0x1A    SET_SEASON (1 = Winter)
    0x1B    SET_SEMSPR_INFLUENCE ( Opentherm )
    0x1D    REFRESH ?
    0x20    OUTSIDE_TEMP  0=none, 1=boiler, 2= web
    0x22    PING
    0x24    STATUS
    0x29    DEVICE_TIME
    0x2A    PROGRAM_END
    0x2B    GET_PROGRAM
    0xFF    Unknowwn

In [ ]:
import { Parser } from "binary-parser";

const BeSmartMessage = new Parser()
    .endianness("little");



const BeSmartPeyload = new Parser()
    .endianness("little")
    .uint8("msgId")
    .bit1("0",{ assert: 0x0 })
    .bit1("uplink")
    .bit1("sync_lost")
    .bit1("3", {assert: 0x0})
    .bit1("downlink")
    .bit1("valid")
    .bit1("write")
    .bit1("response")
    .uint16("meg_length")
    .choice("message",{
        tag: "msgId",
        length: "meg_length",
        choices: {
            0x02: new Parser() // _SetModeMsg, //   SET_MODE
            .endianness("little")
            .nest({
                type: messageDeviceHeader
            })
            .uint8("mode"), //  0=auto 1=manual 2=holiday 3=party 4=off 5=dhw
            0x0A: ProgramMsg, //   PROGRAM ( 1/day )
            0x0B: new Parser() // _SetT3Msg, //   SET_T3
            .endianness("little")
            .nest({
                type: messageDeviceHeader
            })
            .uint16("t3"), //  °C * 10
            0x0C: new Parser() // _SetT2Msg, //   SET_T2
            .endianness("little")
            .nest({
                type: messageDeviceHeader
            })
            .uint16("t2"), // °C * 10
            0x0D: new Parser() // _SetT1Msg, //   SET_T1
            .endianness("little")
            .nest({
                type: messageDeviceHeader
            })
            .uint16("t1"), // °C * 10
            0x11: UnknownMessageIgnore,
//            0x12: _AdvanceMsg, //   ADVANCE 
            0x15: SwVersionMsg, //   SW_VERSION
            0x16: new Parser() // _SetCurvaOTHMsg, //   SET_CURVE ( OpenTherm - °C * 10)
            .endianness("little")
            .nest({
                type: messageDeviceHeader
            })
            .uint8("curve"), 
            0x17: new Parser() //_SetMinHeatSetPointOTHMsg, //   SET_MIN_HEAT_SETP  ( Ophenterm )
            .endianness("little")
            .nest({
                type: messageDeviceHeader
            })
            .uint16("oth_set_min_setpoint"), 
            0x18: new Parser() //_SetMaxHeatSetPointOTHMsg, //  SET_MAX_HEAT_SETP  ( Ophenterm )
            .endianness("little")
            .nest({
                type: messageDeviceHeader
            })
            .uint16("oth_set_max_setpoint"), 
            0x19: new Parser() //_SetUnitMsg, //   SET_UNITS (0=°C 1=°F)
            .endianness("little")
            .nest({
                type: messageDeviceHeader
            })
            .uint8("unit"), //  (0=°C 1=°F)
            0x1A: new Parser() //_SetSeasonMsg, //   SET_SEASON (1 = Winter)
            .endianness("little")
            .nest({
                type: messageDeviceHeader
            })
            .uint8("season"), // (1 = Winter)
            0x1B: new Parser() //_SetSensorInfluenceOTHMsg, //   SET_SEMSPR_INFLUENCE ( Opentherm )
            .endianness("little")
            .nest({
                type: messageDeviceHeader
            })
            .uint8("oth_sensor_influence"), 
            0x1D: new Parser() // RefreshMsg, //   REFRESH ?
            .nest({
                type: messageDeviceHeader
            })    
            .uint32le("unk3")  // ????? optional?
            ,            
            0x20: new Parser()
            .endianness("little")
            .nest({
                type: messageDeviceHeader
            })
            .uint8("val") // 0=no 1=boiler ext tempp 2= web external temp
            , //   OUTSIDE_TEMP  0=none, 1=boiler, 2= web
            0x22: new Parser() //PingMsg, //   PING
            .nest({
                type: messageDeviceHeader
            })    
            .uint16le("unk3",{ asserts: 0xF43C }) // Constants send_PING
            ,
            0x24: StatusMsg, //    STATUS
            0x29: DeviceTimeMsg, //   DEVICE_TIME
            0x2A: ProgramEndMsg, //   PROGRAM_END
            0x2B: new Parser()  //GetProgramMsg, //   GET_PROGRAM
            .nest({
                type: messageRoomHeader
            })
            .uint32le("unk3",{ asserts: 0x800FE0 }) // send_Get_prog
        },
        defaultChoice: UnknownMessage
    })


const BeSmartFrame = new Parser()
    .endianness("little")
    .unit16le("magig_header",{ assert:0xd4fa})
    .unit16le("peyload_lenght")
    .unit32le("sequence")
    .nest("payload",{
        length: "peyload_lenght",
        type: BeSmartPeyload
    })
    .unit16("crc16")
    .unit16("magic_footer",{ assert:0xdf2d});


Specifiche

#
# Note:
#    Downlink (DL) is from cloud server to Besmart device.
#    Uplink (UL) is from Besmart device to cloud server.
#


frame <HHI (LittleEndian, 2u,2u,4u)
2 - 0xd4fa  (Magic Number?) - (unsigned short 2byte LE) - MAGIC_HEADER
2 - peyload length (unisgned short 2byte LE)
4 - sequence number (unisgned int 4byte LE)
(lenght)  payload
2 - 16bit CRC ( unisgned short 2byte LE)
2 - 0xdf2d ( Magic Number?) - (unsigned short 2byte LE) - MAGIC_FOOTER


payload
    1 - (unisgned 1byte LE )msgId
    1 - (1byte LE) Flags
      0  0x0
      1  Uplink ?
      2  Uplink ( sync =1 synclost?)
      3  0x0
      4  Down=1 / Up ( From Cloud To BeSmart / From BeSmart to Cloud )
      5  valid=1 / invalid  
      6  write=1 / read=0
      7  Response
    2 - content Length  (unisgned short 2byte LE)
    (length)  message_content




In [ ]:
const messageHeader = new Parser()
    .endianness("little")
    .unit8le("cseq")
    .unit8le("unk1",{assert: 0x00})
    .uint16le("unk2")
    .uint32le("deviceId")


const messageDeviceHeader = new Parser()
    .endianness("little")
    .nest({
        type: messageHeader
    })
    .uint32le("deviceId")

const messageRoomHeader = new Parser()
    .endianness("little")
    .nest({
        type: messageDeviceHeader
    })
    .uint32le("room")

const StatusMsg = new Parser()
.endianness("little")
.nest({
    type: messageDeviceHeader
})
.array("rooms",{
    length: 8,
    type: new Parser()
        .uint32("room")
        .unit8("byte1")
        //.unit8("byte2")
            .bit1("unk9")
            .bit3("")
            .bit1("mode")
            .bit4("")
        .int16("temp")
        .int16("set_temp")
        .int16("t3")
        .int16("t2")
        .int16("t1")
        .int16("max_setpoint")
        .int16("min_setpoint")
        //.uint8("byte3")
            .bit1("")
            .bit1("advance")
            .bit1("unit")
            .bit1("sensor_influence")
            .bit4("")
        //.unit4("byte4")
            .bit1("winter")
            .bit1("cmd_is_issue")
            .bit1("boost")
            .bit5("")
        .uint16("unk13")
        .uint8("temp_curve")
        .uint8("heating_setpoint")
}
)
// OpenTherm
//.unit8("otFlags1")
    .bit1("")
    .bit1("")
    .bit1("")
    .bit1("")
    .bit1("")
    .bit1("boiler_heating")
    .bit1("dhw_mode")
    .bit1("")
.unit8("otFlags2")
.int16("otUnk1")
.int16("otUnk2")
.int16("tFLO")
.int16("otUnk4")
.int16("tdH")
.int16("tESt")
.int16("otUnk7")
.int16("otUnk8")
.int16("otUnk9")
.int16("otUnk10")
.uint8("wifi_signal")
.uint8("unk16")
.uint16("unk17")
.uint16("unk18")
.uint16("unk19")
.uint16("unk29")

//const GetProgramMsg = new Parser()
//.endianness("little")
//.unit8le("cseq")
//.unit8le("unk1")
//.uint16le("unk2")
//.uint32le("deviceId")
//.uint32le("room")
//.uint32le("unk3")

//const PingMsg = new Parser()
//.endianness("little")
//.unit8le("cseq")
//.unit8le("unk1")
//.uint16le("unk2")
//.uint32le("deviceId")
//.uint16le("unk3")

//const RefreshMsg = new Parser()
//.endianness("little")
//.unit8le("cseq")
//.unit8le("unk1")
//.uint16le("unk2")
//.uint32le("deviceId")

const DeviceTimeMsg = new Parser()
.endianness("little")
.nest({
    type: messageDeviceHeader
})
.uint8("val") // 0=no dst 1=dst
.uint8("unk3")
.uint16("unk4")
.uint32("unk5")



const ProgramMsg = new Parser()
.endianness("little")
.nest({
    type: messageRoomHeader
})
.uint8le("day")
.array("hours",{
    type: "uint8le",
    length: 24
})

const ProgramEndMsg = new Parser()
.endianness("little")
.nest({
    type: messageRoomHeader
})
.uint16le("unk3",{assert: 0x0A14}) // Send

const SwVersionMsg = new Parser()
.endianness("little")
.nest({
    type: messageDeviceHeader
})
.string("version",{ length: 13, stripNull: true })




In [ ]:
    def send_STATUS(self, addr, deviceid, lastseen, response=0):
        cseq = UNUSED_CSEQ
        unk1 = 0x0  # Always zero in DL
        unk2 = 0x0
        payload = struct.pack("<BBHII", cseq, unk1, unk2, deviceid, lastseen)
        wrapper = Wrapper(payload=payload)
        payload = wrapper.encodeDL(MsgId.STATUS, response, write=1)
        logger.info(f"Sending {wrapper}")
        frame = Frame(payload=payload)
        buf = frame.encode()
        logger.info(f"To {addr} {len(buf)} bytes : {hexdump.dump(buf)}")
        self.sendto(buf, addr)

            # OpenTherm parameters
            # From the manual we expect the following to be present somewhere:
f8.8 °C   0...100         # tSEt = set-point flow temperature calculated by the thermostat.
            # tFLO = reading of the boiler flow sensor temperature.
            # trEt = reading of the boiler return sensor temperature.
            # tdH = reading of the boiler DHW sensor temperature.
            # tFLU = reading of the boiler flues sensor temperature.
            # tESt = reading of the boiler outdoor sensor temperature (fitted to the boiler or
            # communicated by the web).
            # MOdU = instantaneous percentage of modulation of boiler fan.
            # FLOr = instantaneous domestic hot water flow rate.
            # HOUr = hours worked in high condensation mode.
            # PrES = central heating system pressure.
            # tFL2 = reading of the heating flow sensor on second circuit

UPD Datagram

https://dgviranmalaka.medium.com/make-udp-communication-more-reliable-in-nodejs-b554acd5c120
